In [25]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [26]:
data = pd.read_csv("Churn_Modelling.csv")
data.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [27]:
# Preprocessing data
data.drop(columns=["RowNumber", "CustomerId", "Surname"],axis=1,inplace=True)
data.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [28]:
# Encoding Categorical Variables
encoder_label_gender = LabelEncoder()
data['Gender'] = encoder_label_gender.fit_transform(data['Gender'])

In [29]:
# One Hot Encoding
encoder_onehot_geography = OneHotEncoder(sparse_output=False)
data['Geography'] = encoder_onehot_geography.fit_transform(data[['Geography']])

In [30]:
# Save the Encoders
with open('labelencoder_gender.pkl','wb') as file:
    pickle.dump(encoder_label_gender, file)

with open('OneHotEncoder_Geography', 'wb') as file:
    pickle.dump(encoder_onehot_geography, file)

In [31]:
# Dividing the dataset into independent and dependent features
X = data.drop(columns=['Exited'], axis=1)
y = data['Exited']

# Split data into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# saving scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Implementation

In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [34]:
# Build our ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL1 connected with input layer
    Dense(32, activation='relu'), # HL2
    Dense(1, activation='sigmoid') # output layer
])

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                704       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2817 (11.00 KB)
Trainable params: 2817 (11.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [37]:
# compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [38]:
# set up tensor board
log_dir = "logs/fit/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [39]:
# set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=5, restore_best_weights=True)

In [40]:
# Training the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
219/219 [==============================] - 1s 3ms/step - loss: 0.4080 - accuracy: 0.8247 - val_loss: 0.3643 - val_accuracy: 0.8613
Epoch 2/100
219/219 [==============================] - 1s 2ms/step - loss: 0.3644 - accuracy: 0.8490 - val_loss: 0.3570 - val_accuracy: 0.8527
Epoch 3/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3577 - accuracy: 0.8503 - val_loss: 0.3493 - val_accuracy: 0.8630
Epoch 4/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3560 - accuracy: 0.8547 - val_loss: 0.3477 - val_accuracy: 0.8597
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3517 - accuracy: 0.8526 - val_loss: 0.3506 - val_accuracy: 0.8597
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3488 - accuracy: 0.8569 - val_loss: 0.3493 - val_accuracy: 0.8600
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3463 - accuracy: 0.8557 - val_loss: 0.3456 - val_accuracy: 0.8623

In [41]:
# saving the model
model.save('model.h5')
# h5 file is compatible with keras that is why we are saving it in h5 format

c:\projects\ANN_Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [42]:
# Launch tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [48]:
%tensorboard --logdir logs/fit/

Reusing TensorBoard on port 6006 (pid 21416), started 1 day, 23:19:16 ago. (Use '!kill 21416' to kill it.)